In [ ]:
%reset

In [1]:
import pandas as pd
import os
import loadmat
%pylab inline
from collections import defaultdict
from scipy import stats
from scipy.signal import hilbert

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import mne

In [2]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta/'
subj, task = ('GP15','EmoGen')

In [3]:
filename = os.path.join(SJdir, 'PCA', 'csvs_FINAL','Bin_Stats_v1_D+R.csv')
df_val = pd.read_csv(filename)
df_val[['RT','t2 (last cross)']].head()


,RT,t2 (last cross)
0,325,212
1,382,228
2,426,341
3,475,389
4,524,406


In [4]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta/'
filename = os.path.join(SJdir, 'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df = pd.read_csv(filename)
df = df[(df.subj == subj) & (df.task == task)]
elecs = df[(df.pattern == 'D') | (df.pattern == 'R')].elec.values

## PLV - phase phase

In [5]:
filename= os.path.join(SJdir, 'Subjs', subj, task, 'lf_hg_phase')
data_dict = loadmat.loadmat(filename)
lf_phase, srate = [data_dict.get(key) for key in ['lf_phase', 'srate']]

In [11]:
if subj in ['GP15', 'GP35']:
    filename = os.path.join(SJdir, 'Subjs', subj, task, 'subj_globals.mat')
    data_dict = loadmat.loadmat(filename)
    original_srate = data_dict['srate'] #1017.3
else:
    original_srate = srate #JH2 is 1000 for both

In [14]:
p = 'D'
filename= os.path.join(SJdir, 'SingleTrials','alltrials','data', 'singletrials_allelecs_smooth_nodecision_' + p + '_dropSR.mat')
data_dict = loadmat.loadmat(filename)
allRTs = data_dict['allRTs']

bins = np.arange(allRTs.min(), allRTs.max(), 50)


In [15]:
#add bin start and stop to df_vals
starts, stops = [[] for i in range(2)]
for i in range(df_val.shape[0]):
    tmp = bins - df_val.iloc[i].RT
    idx = np.where(tmp<0)[0][-1]
    start, stop = bins[idx:idx+2]
    starts.append(start)
    stops.append(stop)

In [16]:
df_val['start'] = starts
df_val['stop'] = stops

df_val[['RT','start','stop','t2 (last cross)']].tail()


,RT,start,stop,t2 (last cross)
38,2229,2200,2250,2105
39,2275,2250,2300,2042
40,2328,2300,2350,2132
41,2525,2500,2550,2105
42,3126,3100,3150,2981


## LF PLV across time - 1 value per trial

In [ ]:
#load in onsets, offsets
filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_word_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_stim = data_dict['onsets_word_corr_g']/original_srate*1000

filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_resp_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_resp = data_dict['onsets_resp_corr_g']/original_srate*1000

#calculate RT, drop outliers
RTs = onsets_resp - onsets_stim
goodidx = (RTs > mean(RTs) - 3* std(RTs)) * (RTs < mean(RTs) + 3 * std(RTs))
RTs = RTs[goodidx]
onsets_stim = onsets_stim[goodidx]
onsets_resp = onsets_resp[goodidx]

#for each trial, calculate which RT bin it falls in, calculate crossover point (make array of crossoverpoints)
bin_idx = np.digitize(RTs, starts)-1
crossovers = df_val.iloc[bin_idx]['t2 (last cross)']

#take relevant window across crossover point - build matrix
lf_c_dict,lf_baseline_dict, lf_pre_c_dict, lf_post_c_dict, = [dict() for i in range(4)]
for e in elecs:    
    lf_c_trials, lf_pre_c_trials, lf_post_c_trials, lf_baseline_trials = [np.empty((len(onsets_stim),250)) for i in range(4)]
    
    for i, c in enumerate(crossovers.values):
        c_point = int((onsets_stim[i]+c))
        lf_c_trials[i,:] = lf_phase[e, c_point - 125 : c_point + 125]
        lf_pre_c_trials[i,:] = lf_phase[e, c_point - 250 : c_point]
        lf_post_c_trials[i,:] = lf_phase[e, c_point : c_point + 250]
        lf_baseline_trials[i,:] = lf_phase[e, onsets_stim[i] - 250 : onsets_stim[i]]
        
    lf_c_dict[e] = lf_c_trials    
    lf_pre_c_dict[e] = lf_pre_c_trials
    lf_post_c_dict[e] = lf_post_c_trials
    lf_baseline_dict[e] = lf_baseline_trials


In [ ]:
#calculate pac - one value per trial (mean across time)
tmp = pd.DataFrame()

for e1 in elecs:
    for e2 in elecs:
        if e1<=e2:
            pattern = '%s-%s' %(df[df.elec == e1].pattern.values[0], df[df.elec == e2].pattern.values[0])

            f, ax = plt.subplots(2, 2, subplot_kw=dict(projection='polar'), figsize = (10,10))
            plt.suptitle('%s %s PLV : e%i (LF) - e%i (HG) : %s' %(subj, task, e1, e2, pattern))
            ax = ax.flatten()

            plv_c, plv_pre_c, plv_post_c, plv_baseline = [[] for i in range(4)]
            for i in range(len(onsets_stim)): #per trial
                plv_c.append(lf_c_dict[e1][i,:] - lf_c_dict[e2][i,:])
                plv_pre_c.append(lf_pre_c_dict[e1][i,:] - lf_pre_c_dict[e2][i,:])
                plv_post_c.append(lf_post_c_dict[e1][i,:] - lf_post_c_dict[e2][i,:])
                plv_baseline.append(lf_baseline_dict[e1][i,:] - lf_baseline_dict[e2][i,:])

            plv_c = np.mean(exp(1j * np.array(plv_c)), 1) #1 complex value per trial
            plv_pre_c = np.mean(exp(1j * np.array(plv_pre_c)), 1)
            plv_post_c = np.mean(exp(1j * np.array(plv_post_c)), 1)
            plv_baseline = np.mean(exp(1j * np.array(plv_baseline)), 1)

            for i in plv_baseline:
                ax[0].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
            ax[0].plot([0, mean(angle(plv_baseline))], [0, mean(abs(plv_baseline))], color = 'k', lw = 4)
            ax[0].set_title("baseline = %.2f" %(mean(abs(plv_baseline))))

            for i in plv_pre_c:
                ax[1].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
            ax[1].plot([0, (mean(angle(plv_pre_c)))], [0, mean(abs(plv_pre_c))], color = 'k', lw = 4)
            ax[1].set_title("pre cross = %.2f" %(mean(abs(plv_pre_c))))

            for i in plv_c: #per trial
                ax[2].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
            ax[2].plot([0, (mean(angle(plv_c)))], [0, mean(abs(plv_c))], color = 'k', lw = 4)
            ax[2].set_title("crossover = %.2f" %(mean(abs(plv_c))))

            for i in plv_post_c:
                ax[3].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
            ax[3].plot([0, mean(angle(plv_post_c))], [0, mean(abs(plv_post_c))], color = 'k', lw = 4)
            ax[3].set_title("post cross = %.2f" %(mean(abs(plv_post_c))))

            filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/LF_PLV/figures/%s_%s_e%i_e%i_trials.png' %(subj, task, e1, e2)
            f.savefig(filename, bbox_inches='tight')
            plt.close()


            tmp = tmp.append({'LF_elec1': e1, 'LF_elec2' : e2, 'window': 'baseline', 'plv' : mean(abs(plv_baseline)), 'p': 1}, ignore_index = True)

            _, pval = stats.ttest_ind(np.arctanh(abs(plv_baseline)), np.arctanh(abs(plv_pre_c)))
            tmp = tmp.append({'LF_elec1': e1, 'LF_elec2' : e2, 'window': 'pre', 'plv' : mean(abs(plv_pre_c)), 'p': pval}, ignore_index = True)

            _, pval = stats.ttest_ind(np.arctanh(abs(plv_baseline)), np.arctanh(abs(plv_c)))
            tmp = tmp.append({'LF_elec1': e1, 'LF_elec2' : e2, 'window': 'cross', 'plv' : mean(abs(plv_c)), 'p': pval}, ignore_index = True)

            _, pval = stats.ttest_ind(np.arctanh(abs(plv_baseline)), np.arctanh(abs(plv_post_c)))
            tmp = tmp.append({'LF_elec1': e1, 'LF_elec2' : e2, 'window': 'post', 'plv' : mean(abs(plv_post_c)), 'p': pval}, ignore_index = True)

filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/LF_PLV/%s_%s_plv_trials.csv' %(subj, task)
tmp.to_csv(filename, index = False)
filename

## PLV across trials - 1 value per time point

In [17]:
#load in onsets, offsets
filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_word_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_stim = data_dict['onsets_word_corr_g']/original_srate*1000

filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_resp_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_resp = data_dict['onsets_resp_corr_g']/original_srate*1000

#calculate RT, drop outliers
RTs = onsets_resp - onsets_stim
goodidx = (RTs > mean(RTs) - 3* std(RTs)) * (RTs < mean(RTs) + 3 * std(RTs))
RTs = RTs[goodidx]
onsets_stim = onsets_stim[goodidx]
onsets_resp = onsets_resp[goodidx]

#for each trial, calculate which RT bin it falls in, calculate crossover point (make array of crossoverpoints)
bin_idx = np.digitize(RTs, starts)-1
crossovers = df_val.iloc[bin_idx]['t2 (last cross)']

#take relevant window across crossover point - build matrix
lf_c_dict,lf_baseline_dict,lf_pre_c_dict, lf_post_c_dict = [dict() for i in range(4)]
for e in elecs:    
    lf_c_trials, hg_c_trials = [np.empty((len(onsets_stim),1000)) for i in range(2)]
    
    for i, c in enumerate(crossovers.values):
        c_point = int((onsets_stim[i]+c))
        lf_c_trials[i,:] = lf_phase[e, c_point - 500 : c_point + 500]        
    lf_c_dict[e] = lf_c_trials


In [19]:
#calculate plv - one value per timepoint (mean across trials, 1000 points)
tmp = pd.DataFrame()

for e1 in elecs:
    for e2 in elecs:
        if e1<e2:
            pattern = '%s-%s' %(df[df.elec == e1].pattern.values[0], df[df.elec == e2].pattern.values[0])

            phase_diff = list()
            for i in range(len(onsets_stim)): #per trial
                phase_diff.append(lf_c_dict[e1][i,:] - lf_c_dict[e2][i,:])

            plv_c = np.mean(exp(1j * np.array(phase_diff)), 0) #average across trials, get time series

            plv_c_z = (abs(plv_c) - np.mean(abs(plv_c))) / np.std(abs(plv_c))

            #tmp = tmp.append(pd.DataFrame({'LF_elec': [e1] *1000, 'HG_elec' : [e2] * 1000, 'pattern': [pattern] * 1000, 'plv' : pd.Series(abs(plv_c), name = 'plv', index = np.arange(-500,500))}))
            tmp = tmp.append(pd.DataFrame({'LF_elec1': [e1] *1000, 'LF_elec2' : [e2] * 1000, 'pattern': [pattern] * 1000, 'plv' : pd.Series(plv_c_z, name = 'plv', index = np.arange(-500,500))}))
            f, ax = plt.subplots()
            ax.plot(range(-500, 500), abs(plv_c))
            ax.set_xlim([-500, 500])
            ax.set_ylim([-0.1, 1])

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.get_xaxis().tick_bottom()
            ax.get_yaxis().tick_left()
            ax.tick_params( axis='both', which = 'both', top = 'off', bottom = 'off', right = 'off', left = 'off')

            ax.axvline(0, color = 'k')
            ax.axhline(0, color = 'k')

            filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/LF_PLV/figures/time/%s_%s_e%i_e%i_time.png' %(subj, task, e1, e2)
            f.savefig(filename, bbox_inches='tight')
            plt.close()

df_zscore = tmp.reset_index()
df_zscore.columns = ['time','LF_elec1','LF_elec2','pattern','plv'] #zscored

# PAC

In [ ]:
filename= os.path.join(SJdir, 'Subjs', subj, task, 'lf_hg_phase')
data_dict = loadmat.loadmat(filename)
hg_phase, lf_phase, srate = [data_dict.get(key) for key in ['hg_phase','lf_phase', 'srate']]

In [ ]:
filename = os.path.join(SJdir, 'Subjs', subj ,task, 'gdat_notch')
data_dict = loadmat.loadmat(filename)
gdat = data_dict['gdat']


In [ ]:
if subj in ['GP15', 'GP35']:
    filename = os.path.join(SJdir, 'Subjs', subj, task, 'subj_globals.mat')
    data_dict = loadmat.loadmat(filename)
    original_srate = data_dict['srate'] #1017.3
else:
    original_srate = srate #JH2 is 1000 for both

In [ ]:
original_srate

In [ ]:
f1, f2 = (7,11)

In [ ]:
# Slow phase time series
phs_bp = mne.filter.band_pass_filter(gdat, srate, f1, f2)
phs_ang = np.angle(hilbert(phs_bp))

In [ ]:
# Fast phase time series
amp_bp = mne.filter.band_pass_filter(gdat, srate, 70, 150)
amp_env = np.absolute(hilbert(amp_bp))
amp_env_bp = mne.filter.band_pass_filter(amp_env, srate, f1, f2)
amp_env_ang = np.angle(hilbert(amp_env_bp))

In [ ]:
filename = os.path.join(SJdir, 'PCA', 'csvs_FINAL','Bin_Stats_v1_D+R.csv')
df_val = pd.read_csv(filename)
df_val[['RT','t2 (last cross)']].head()


In [ ]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta/'
filename = os.path.join(SJdir, 'PCA', 'csvs_FINAL', 'mean_traces_all_subjs_dropSR.csv')
df = pd.read_csv(filename)
df = df[(df.subj == subj) & (df.task == task)]
elecs = df[(df.pattern == 'D') | (df.pattern == 'R')].elec.values

In [ ]:
p = 'D'
filename= os.path.join(SJdir, 'SingleTrials','alltrials','data', 'singletrials_allelecs_smooth_nodecision_' + p + '_dropSR.mat')
data_dict = loadmat.loadmat(filename)
allRTs = data_dict['allRTs']

bins = np.arange(allRTs.min(), allRTs.max(), 50)


In [ ]:
#add bin start and stop to df_vals
starts, stops = [[] for i in range(2)]
for i in range(df_val.shape[0]):
    tmp = bins - df_val.iloc[i].RT
    idx = np.where(tmp<0)[0][-1]
    start, stop = bins[idx:idx+2]
    starts.append(start)
    stops.append(stop)

In [ ]:
df_val['start'] = starts
df_val['stop'] = stops

df_val[['RT','start','stop','t2 (last cross)']].tail()


## PAC across time - plot 1 value per trial

In [ ]:
#load in onsets, offsets

filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_word_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_stim = data_dict['onsets_word_corr_g']/original_srate*1000

filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_resp_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_resp = data_dict['onsets_resp_corr_g']/original_srate*1000

#calculate RT, drop outliers
RTs = onsets_resp - onsets_stim
goodidx = (RTs > mean(RTs) - 3* std(RTs)) * (RTs < mean(RTs) + 3 * std(RTs))
RTs = RTs[goodidx]
onsets_stim = onsets_stim[goodidx]
onsets_resp = onsets_resp[goodidx]

#for each trial, calculate which RT bin it falls in, calculate crossover point (make array of crossoverpoints)
bin_idx = np.digitize(RTs, starts)-1
crossovers = df_val.iloc[bin_idx]['t2 (last cross)']

#take relevant window across crossover point - build matrix
lf_c_dict, hg_c_dict, lf_baseline_dict, hg_baseline_dict, lf_pre_c_dict, lf_post_c_dict, hg_pre_c_dict, hg_post_c_dict = [dict() for i in range(8)]
for e in elecs:    
    lf_c_trials, hg_c_trials, lf_pre_c_trials, hg_pre_c_trials, lf_post_c_trials, hg_post_c_trials, lf_baseline_trials, hg_baseline_trials = [np.empty((len(onsets_stim),250)) for i in range(8)]
    
    for i, c in enumerate(crossovers.values):
        c_point = int((onsets_stim[i]+c))
        lf_c_trials[i,:] = lf_phase[e, c_point - 125 : c_point + 125]
        hg_c_trials[i,:] = hg_phase[e, c_point-125 : c_point + 125]
        
        lf_pre_c_trials[i,:] = lf_phase[e, c_point - 250 : c_point]
        hg_pre_c_trials[i,:] = hg_phase[e, c_point - 250 : c_point]
        
        lf_post_c_trials[i,:] = lf_phase[e, c_point : c_point + 250]
        hg_post_c_trials[i,:] = hg_phase[e, c_point : c_point + 250]
        
        lf_baseline_trials[i,:] = lf_phase[e, onsets_stim[i] - 250 : onsets_stim[i]]
        hg_baseline_trials[i,:] = hg_phase[e, onsets_stim[i] - 250 : onsets_stim[i]]
        
    lf_c_dict[e] = lf_c_trials
    hg_c_dict[e] = hg_c_trials
    
    lf_pre_c_dict[e] = lf_pre_c_trials
    hg_pre_c_dict[e] = hg_pre_c_trials
    
    lf_post_c_dict[e] = lf_post_c_trials
    hg_post_c_dict[e] = hg_post_c_trials
    
    lf_baseline_dict[e] = lf_baseline_trials
    hg_baseline_dict[e] = hg_baseline_trials

In [ ]:
#calculate pac - one value per trial (mean across time)
tmp = pd.DataFrame()

for e1 in elecs:
    for e2 in elecs:
        pattern = '%s-%s' %(df[df.elec == e1].pattern.values[0], df[df.elec == e2].pattern.values[0])
        
        f, ax = plt.subplots(2, 2, subplot_kw=dict(projection='polar'), figsize = (10,10))
        plt.suptitle('%s %s PLV : e%i (LF) - e%i (HG) : %s' %(subj, task, e1, e2, pattern))
        ax = ax.flatten()

        plv_c, plv_pre_c, plv_post_c, plv_baseline = [[] for i in range(4)]
        for i in range(len(onsets_stim)): #per trial
            plv_c.append(lf_c_dict[e1][i,:] - hg_c_dict[e2][i,:])
            plv_pre_c.append(lf_pre_c_dict[e1][i,:] - hg_pre_c_dict[e2][i,:])
            plv_post_c.append(lf_post_c_dict[e1][i,:] - hg_post_c_dict[e2][i,:])
            plv_baseline.append(lf_baseline_dict[e1][i,:] - hg_baseline_dict[e2][i,:])
            
        plv_c = np.mean(exp(1j * np.array(plv_c)), 1) #1 complex value per trial
        plv_pre_c = np.mean(exp(1j * np.array(plv_pre_c)), 1)
        plv_post_c = np.mean(exp(1j * np.array(plv_post_c)), 1)
        plv_baseline = np.mean(exp(1j * np.array(plv_baseline)), 1)
        
        for i in plv_baseline:
            ax[0].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
        ax[0].plot([0, mean(angle(plv_baseline))], [0, mean(abs(plv_baseline))], color = 'k', lw = 4)
        ax[0].set_title("baseline = %.2f" %(mean(abs(plv_baseline))))

        for i in plv_pre_c:
            ax[1].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
        ax[1].plot([0, (mean(angle(plv_pre_c)))], [0, mean(abs(plv_pre_c))], color = 'k', lw = 4)
        ax[1].set_title("pre cross = %.2f" %(mean(abs(plv_pre_c))))

        for i in plv_c: #per trial
            ax[2].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
        ax[2].plot([0, (mean(angle(plv_c)))], [0, mean(abs(plv_c))], color = 'k', lw = 4)
        ax[2].set_title("crossover = %.2f" %(mean(abs(plv_c))))
        
        for i in plv_post_c:
            ax[3].plot([0, i.real], [0, i.imag], color = 'b', alpha = 0.5)
        ax[3].plot([0, mean(angle(plv_post_c))], [0, mean(abs(plv_post_c))], color = 'k', lw = 4)
        ax[3].set_title("post cross = %.2f" %(mean(abs(plv_post_c))))
        
        filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/figures/%s_%s_e%i_e%i_trials.png' %(subj, task, e1, e2)
        f.savefig(filename, bbox_inches='tight')
        plt.close()


        tmp = tmp.append({'LF_elec': e1, 'HG_elec' : e2, 'window': 'baseline', 'plv' : mean(abs(plv_baseline)), 'p': 1}, ignore_index = True)

        _, pval = stats.ttest_ind(np.arctanh(abs(plv_baseline)), np.arctanh(abs(plv_pre_c)))
        tmp = tmp.append({'LF_elec': e1, 'HG_elec' : e2, 'window': 'pre', 'plv' : mean(abs(plv_pre_c)), 'p': pval}, ignore_index = True)

        _, pval = stats.ttest_ind(np.arctanh(abs(plv_baseline)), np.arctanh(abs(plv_c)))
        tmp = tmp.append({'LF_elec': e1, 'HG_elec' : e2, 'window': 'cross', 'plv' : mean(abs(plv_c)), 'p': pval}, ignore_index = True)

        _, pval = stats.ttest_ind(np.arctanh(abs(plv_baseline)), np.arctanh(abs(plv_post_c)))
        tmp = tmp.append({'LF_elec': e1, 'HG_elec' : e2, 'window': 'post', 'plv' : mean(abs(plv_post_c)), 'p': pval}, ignore_index = True)

filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/%s_%s_plv_trials.csv' %(subj, task)
#tmp.to_csv(filename, index = False)
filename

In [ ]:
# v = pd.DataFrame({'raw complex': plv_c, 'absolute' : abs(plv_c),  'fisher transform' : np.arctanh(abs(plv_c))})
# v = v[['raw complex','absolute', 'fisher transform']]
# v.to_csv('/home/knight/matar/JH2_e2_e3_PAC.csv', index = False)

## PAC across trials - 1 value per time point

In [ ]:
original_srate

In [ ]:
#load in onsets, offsets
filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_word_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_stim = data_dict['onsets_word_corr_g']/original_srate*1000

filename= os.path.join(SJdir, 'Subjs', subj, task, 'onsets_resp_corr_g')
data_dict = loadmat.loadmat(filename)
onsets_resp = data_dict['onsets_resp_corr_g']/original_srate*1000

#calculate RT, drop outliers
RTs = onsets_resp - onsets_stim
goodidx = (RTs > mean(RTs) - 3* std(RTs)) * (RTs < mean(RTs) + 3 * std(RTs))
RTs = RTs[goodidx]
onsets_stim = onsets_stim[goodidx]
onsets_resp = onsets_resp[goodidx]

#for each trial, calculate which RT bin it falls in, calculate crossover point (make array of crossoverpoints)
bin_idx = np.digitize(RTs, starts)-1
crossovers = df_val.iloc[bin_idx]['t2 (last cross)']

#take relevant window across crossover point - build matrix
lf_c_dict, hg_c_dict, lf_baseline_dict, hg_baseline_dict, lf_pre_c_dict, lf_post_c_dict, hg_pre_c_dict, hg_post_c_dict = [dict() for i in range(8)]
for e in elecs:    
    lf_c_trials, hg_c_trials = [np.empty((len(onsets_stim),1000)) for i in range(2)]
    
    for i, c in enumerate(crossovers.values):
        c_point = int((onsets_stim[i]+c))
        lf_c_trials[i,:] = lf_phase[e, c_point - 500 : c_point + 500]
        hg_c_trials[i,:] = hg_phase[e, c_point - 500 : c_point + 500]
        
    lf_c_dict[e] = lf_c_trials
    hg_c_dict[e] = hg_c_trials

In [ ]:
#calculate plv - one value per timepoint (mean across trials, 1000 points)
tmp = pd.DataFrame()

for e1 in elecs:
    for e2 in elecs:
        pattern = '%s-%s' %(df[df.elec == e1].pattern.values[0], df[df.elec == e2].pattern.values[0])

        phase_diff = list()
        for i in range(len(onsets_stim)): #per trial
            phase_diff.append(lf_c_dict[e1][i,:] - hg_c_dict[e2][i,:])
            
        plv_c = np.mean(exp(1j * np.array(phase_diff)), 0) #average across trials, get time series
        
        plv_c_z = (abs(plv_c) - np.mean(abs(plv_c))) / np.std(abs(plv_c))
        
        #tmp = tmp.append(pd.DataFrame({'LF_elec': [e1] *1000, 'HG_elec' : [e2] * 1000, 'pattern': [pattern] * 1000, 'plv' : pd.Series(abs(plv_c), name = 'plv', index = np.arange(-500,500))}))
        tmp = tmp.append(pd.DataFrame({'LF_elec': [e1] *1000, 'HG_elec' : [e2] * 1000, 'pattern': [pattern] * 1000, 'plv' : pd.Series(plv_c_z, name = 'plv', index = np.arange(-500,500))}))
#         f, ax = plt.subplots()
#         ax.plot(range(-500, 500), abs(plv_c))
#         ax.set_xlim([-500, 500])
#         ax.set_ylim([-0.1, 1])

#         ax.spines['top'].set_visible(False)
#         ax.spines['right'].set_visible(False)
#         ax.get_xaxis().tick_bottom()
#         ax.get_yaxis().tick_left()
#         ax.tick_params( axis='both', which = 'both', top = 'off', bottom = 'off', right = 'off', left = 'off')
        
#         ax.axvline(0, color = 'k')
#         ax.axhline(0, color = 'k')
        
#         filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/figures/time/%s_%s_e%i_e%i_time.png' %(subj, task, e1, e2)
#         f.savefig(filename, bbox_inches='tight')
#         plt.close()

In [ ]:
df_zscore = tmp.reset_index()
df_zscore.columns = ['time','HG_elec','LF_elec','pattern','plv'] #zscored

## stats on 300 ms flanking crossover point

In [ ]:
# df_pre = df_zscore[(df_zscore.time<=0) & (df_zscore.time>-300)]
# df_post = df_zscore[(df_zscore.time<300) & (df_zscore.time>=0)]
dftmp = pd.DataFrame()
for e1 in elecs:
    for e2 in elecs:
        #pattern = '%s-%s' %(df_zscore[df_zscore.HG_elec == e1].pattern.values[0], df_zscore[df_zscore.LF_elec == e2].pattern.values[0])
        #pattern = np.unique(df_pre.pattern)[0]
        #pre = df_pre[(df_pre.HG_elec == e1) & (df_pre.LF_elec == e2)].plv
        #post = df_post[(df_post.HG_elec == e1) & (df_post.LF_elec == e2)].plv
        df_plv = df_zscore[(df_zscore.HG_elec == e1) & (df_zscore.LF_elec == e2)]
        pattern = np.unique(df_plv.pattern)[0]
        pre = df_plv[(df_plv.time<=0) & (df_plv.time>-300)].plv
        post = df_plv[(df_plv.time<300) & (df_plv.time>=0)].plv
        t, pval = stats.ttest_ind(pre, post)
        dftmp = dftmp.append({'LF_elec': e1, 'HG_elec' : e2, 'pattern': pattern, 'mean_pre' : pre.mean(), 'mean_post': post.mean(), 'p' : pval, 't': t}, ignore_index = True)

filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/plv_signifiance_crossover_300ms_zscore_%s_%s.csv' %(subj, task)
dftmp.to_csv(filename)   
filename

In [ ]:
dftmp[dftmp.p<0.0001].groupby('pattern').count().p/dftmp.groupby('pattern').count().p

## stats on mean traces
- mean of pre/post window per pair
- ttest of all the means per pattern
- output mean, t, p for each pattern

In [ ]:
dftmp = pd.DataFrame()
for pattern in ['D-D','D-R','R-R','R-D']:
    pre, post = [[] for i in range(2)]
    for e1 in elecs:
        for e2 in elecs:
            df_plv = df_zscore[(df_zscore.HG_elec == e1) & (df_zscore.LF_elec == e2) & (df_zscore.pattern == pattern)]
            pre.append(df_plv[(df_plv.time<=0) & (df_plv.time>-300)].plv.mean())
            post.append(df_plv[(df_plv.time<300) & (df_plv.time>=0)].plv.mean())
    
            pre = [x for x in pre if str(x) != 'nan']    
            post = [x for x in post if str(x) != 'nan'] 

    t, pval = stats.ttest_ind(pre, post)
    dftmp = dftmp.append({'pattern': pattern, 'mean_pre' : np.nanmean(pre), 'mean_post': np.nanmean(post), 'p' : pval, 't': t}, ignore_index = True)

In [ ]:
print dftmp

In [ ]:
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/plv_signifiance_crossover_300_time_mean_%s_%s.csv' %(subj, task)
dftmp.to_csv(filename)

In [ ]:
dftmp[dftmp.p<0.0001].groupby('pattern').count().p/dftmp.groupby('pattern').count().p

## individual traces

In [ ]:
counts = df_zscore.groupby('pattern').count()/1000
f, ax = plt.subplots()
# ax = df_zscore[df_zscore.pattern == 'D-D'].groupby('time').plv.mean().plot(label = 'D-D, %i' %(counts.loc['D-D'].time))
# df_zscore[df_zscore.pattern == 'D-R'].groupby('time').plv.mean().plot(label = 'D-R, %i' %(counts.loc['D-R'].time))
# df_zscore[df_zscore.pattern == 'R-D'].groupby('time').plv.mean().plot(label = 'R-D, %i' %(counts.loc['R-D'].time))
for x in [('D-D','b'),('D-R','r'),('R-D','g')]:
    pattern, c = x
    n = df_zscore[df_zscore.pattern == pattern].groupby('time').plv.count().iloc[0]
    plv_mean = df_zscore[df_zscore.pattern == pattern].groupby('time').plv.mean()
    plv_sem = df_zscore[df_zscore.pattern == pattern].groupby('time').plv.std()/n
    ax.plot(range(-500,500), plv_mean, label = '%s %i' %(pattern, n), color = c)
    ax.fill_between(range(-500,500), plv_mean+plv_sem, plv_mean-plv_sem, alpha = 0.5, edgecolor = 'None', facecolor = c, label = None)


plt.legend()
ax.set_title('%s %s' %(subj, task))
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/%s_%s_plv_time_pattern_avg.png' %(subj, task)
#plt.savefig(filename)

In [ ]:
f,ax = plt.subplots()
pattern = 'D-D'
tmp_pattern = tmp[tmp.pattern == pattern]
HG = np.unique(tmp_pattern['HG_elec'])
LF = np.unique(tmp_pattern['LF_elec'])

for e1 in HG:
    for e2 in LF:
        tmp_pattern[(tmp_pattern.HG_elec == e1) & (tmp_pattern.LF_elec == e2)].plv.plot(alpha = 0.5)
ax.set_title('%s %s %s' %(subj, task, pattern))

## run on all subj/tasks

In [ ]:
import crossover_analysis as ca

In [ ]:
subj_dict = {'GP15': ['FaceGen','SelfAud','SelfVis', 'EmoRep','EmoGen']}
subj_dict['GP35'] = ['FaceEmo','FaceGen', 'EmoRep',]
subj_dict['JH2'] = ['SelfAud','SelfVis']

for subj in subj_dict.keys():
    for task in subj_dict[subj]:
        print(subj, task)
        ca.crossover_by_trials(subj, task)
        
        

In [ ]:
import glob

In [ ]:
#combine csv
df_full = pd.DataFrame()
files = glob.glob(os.path.join(SJdir, 'PCA','PLV', 'mean_traces', '*.csv'))
for f in files:
    df = pd.read_csv(f)
    df['subj'] = f.split('_')[-2]
    df['task'] = f.split('_')[-1][:-4]
    df_full = df_full.append(df)

In [ ]:
df_full = df_full[['subj','task','pattern', 'mean_post','mean_pre','t','p']]

filename = os.path.join(SJdir, 'PCA','PLV', 'mean_traces', 'all_subjects_plv_signifiance_crossover_300_time_mean.csv')
df_full.to_csv(filename, index = False)

## summaries

In [ ]:
files = glob.glob(os.path.join(SJdir, 'PCA','PLV', 'flanking_300ms', '*.csv'))
for f in files:
    df = pd.read_csv(f)
    dfmean = df.groupby('pattern').mean()[['mean_post', 'mean_pre', 'p','t']]
    dfstd= df.groupby('pattern').std()[['mean_post', 'mean_pre', 'p','t']]
    dfstd.columns = [x+'_std' for x in dfstd.columns]
    df = pd.concat([dfmean, dfstd], axis = 1)
    filename = f.split('.')[0]+'_summary.csv'
    df.to_csv(filename)    

In [ ]:
df_full = pd.DataFrame()
files = glob.glob(os.path.join(SJdir, 'PCA','PLV', 'flanking_300ms', '*_summary.csv'))
files = [x for x in files if 'all' not in x]
for f in files:
    df = pd.read_csv(f)
    subj, task = f.split('_')[-3:-1]
    df['subj'] = subj
    df['task'] = task
    df_full = df_full.append(df)
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/flanking_300ms/plv_signifiance_crossover_300ms_zscore_all_subjs_summary.csv'
df_full.to_csv(filename, index = False)

In [ ]:
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/%s_%s_plv.csv' %('GP35', task)
df = pd.read_csv(filename)

df[df.pvalue<0.01].groupby(['window', 'pattern']).count()['pvalue'] / df[df['window']!='baseline'].groupby(['window','pattern']).count()['pvalue']

In [ ]:
pd.pivot_table(gp35[gp35.pvalue<0.01], values = 'pvalue', index = 'window', columns = 'pattern', aggfunc='count') / pd.pivot_table(gp35, values = 'pvalue', index = 'window', columns = 'pattern', aggfunc='count')

In [ ]:
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/%s_%s_plv.csv' %('GP15', task)
df = pd.read_csv(filename)

df[df.pvalue<0.01].groupby(['window', 'pattern']).count()['pvalue'] / df[df['window']!='baseline'].groupby(['window','pattern']).count()['pvalue']

In [ ]:
filename = '/home/knight/matar/MATLAB/DATA/Avgusta/PCA/PLV/%s_%s_plv.csv' %('JH2', task)
df = pd.read_csv(filename)

df[df.pvalue<0.01].groupby(['window', 'pattern']).count()['pvalue'] / df[df['window']!='baseline'].groupby(['window','pattern']).count()['pvalue']

In [ ]:
df[df.pvalue<0.01].groupby(['window', 'pattern']).count()['pvalue']

In [ ]:
f, ax = plt.subplots(figsize = (10,10))
f = ax.imshow(log10(mtx), interpolation='nearest')
plt.colorbar(f)

In [ ]:
for e1 in [5]:
    for e2 in elecs:
        f, ax = plt.subplots()
        ax.hist(np.arctanh(plv[e1,e2]))
        ax.hist(np.arctanh(plv_baseline[e1,e2]))

In [ ]:
(RTs - crossovers.values).mean() #average offset between RT and crossover point
